In [1]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 83c7a2dd-bef5-47dc-bc6b-3a4b432d84e8

In [2]:
import vision as vis
import kalman
import numpy as np
import time
import cv2 as cv
import shortest_path as short
import positions as pos
import math
import motion_control as motion



COMPUTE = 0
TRAVEL = 1
LOCAL_NAV = 2
FINISH = 3

In [ ]:
# Main -> state machine
state = COMPUTE
obstacle = False
path_calculated = False

#valeurs pour initialiser
robot_pos_kalman = np.array([[0],[0],[0]]) #3-by-1 np.array
Sigma_kalman = np.diag([0.01,0.01,0.01])
HISTORY_SIZE = 15
history_kalman = [np.reshape(robot_pos_kalman,(1,3))]*HISTORY_SIZE

cap = cv.VideoCapture(2)  
cap.set(cv.CAP_PROP_AUTO_EXPOSURE, 1)
while True:
    if state == COMPUTE:
        captured, img = cap.read()
        if captured:
            cropped, cropped_img = vis.crop_map(img)
            # print(cropped_img.shape)        # uncomment to debug
            # cv.imshow("Original", img)        # uncomment to debug
            cv.imshow("Cropped", cropped_img)
            if cropped:
                im3 = np.zeros(cropped_img.shape)
                found, obstacles = vis.extract_obstacles(cropped_img)
                if found:
                    ex_obstacles = vis.expand_obstacles(obstacles)
                    cv.drawContours(im3, ex_obstacles, -1, (0, 255, 0), 3)
                    cv.imshow("expanded", im3)
                found_rob, robot_pos = pos.get_robot_position(cropped_img)
                if found_rob:
                    vis.annotate_robot(robot_pos, im3)
                found_goal, goal_pos, radius_pxl = pos.get_goal_position(cropped_img)
                if found_goal:
                    vis.annotate_goal(goal_pos, im3)
                found_arch, arch_pos = pos.get_arch_positions(cropped_img)
                if found_arch and found_rob:
                    if math.dist(arch_pos[0],robot_pos[0])>math.dist(arch_pos[1],robot_pos[0]):
                        arch_pos = np.flip(arch_pos,0)
                    vis.annotate_arch(arch_pos, im3)
                if (found and found_rob and found_goal and found_arch):
                    scale_factor = pos.convert_meter2pxl(radius_pxl)
                    AXLE_LENGTH = 0.095*scale_factor #used for kalman filter
                    if path_calculated == False:
                        vertices = vis.convert_vertice(ex_obstacles)
                        pathname = short.find_shortest_path(vertices, tuple(robot_pos[0]), tuple(arch_pos[0]))
                        path_arch =\
                        short.names_to_subpaths(pathname,vertices,tuple(robot_pos[0]), tuple(arch_pos[0]))[0]
                        vis.annotate_path(path_arch,im3)

                        pathname2 = short.find_shortest_path(vertices, tuple(arch_pos[1]), tuple(goal_pos))
                        path_goal =\
                        short.names_to_subpaths(pathname2,vertices,tuple(arch_pos[1]), tuple(goal_pos))[0]
                        vis.annotate_path(path_goal,im3)
                        cv.imshow("environment", im3)
                        
                        #initialize kalman filter
                        robot_pos_kalman = np.array([[robot_pos[0][0]],[robot_pos[0][1]],[robot_pos[1]]]) #3-by-1 np.array
                        HISTORY_SIZE = 10
                        history_kalman = [np.reshape(robot_pos_kalman,(1,3))]*HISTORY_SIZE
                        Ts = 0.15 #initialize
                        
                        state = TRAVEL
        else:
            print("There was a problem in the capture")
            break
    if state == TRAVEL:
        cv.imshow("environment", im3)
        cv.waitKey(0)
        for subgoal in path_arch[1:]:
            while not motion.check_robot_arrived(robot_pos,subgoal):
                captured, img_travel = cap.read()
                cropped, cropped_travel = vis.crop_map(img_travel)
                cv.imshow("cropped",cropped_travel)
                im4 = np.zeros(cropped_travel.shape)
                vis.annotate_robot(robot_pos,im4)
                vis.annotate_path(path_arch,im4)
                
                
                #read values from camera & sensors
                success_camera, robot_pos_camera = pos.get_robot_position(cropped_travel)
                #print("robot angle", robot_pos_camera[1])
                if success_camera:
                    camera_input = [(robot_pos_camera[0][0],robot_pos_camera[0][1]), robot_pos_camera[1], success_camera]
                    print("seen by camera", camera_input)
                else:
                    camera_input = [(0,0),0,False]
                await node.wait_for_variables({"motor.left.speed"})
                await node.wait_for_variables({"motor.right.speed"})
                motorspeed = [node.v.motor.right.speed, node.v.motor.left.speed]        
                
                
                #get position from kalman filter
                robot_pos_kalman, Sigma_kalman, history_kalman =\
                kalman.kalmanfilter(robot_pos_kalman,Sigma_kalman,motorspeed,history_kalman, camera_input, Ts, scale_factor, AXLE_LENGTH)
                robot_pos = [(int(robot_pos_kalman[0][0]), int(robot_pos_kalman[1][0])), robot_pos_kalman[2][0]]
                print("robot_pos",robot_pos)
                print("kalman[0][0]", robot_pos_kalman[0][0])
                print("kalman[1][0]", robot_pos_kalman[1][0])
                print("kalman[2][0]", robot_pos_kalman[2][0])
                #et apres, on passe robot_pos au controleur
                
                
                err = motion.get_error(robot_pos,subgoal)
                cv.putText(im4, str(err), (40, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv.imshow("mission status", im4)
                cv.waitKey(10)
                motor_left, motor_right = motion.speed_control(err)
                motor_left = round(motor_left)
                motor_right = round(motor_right)
                v = {
                    "motor.left.target": [motor_left],
                    "motor.right.target": [motor_right],
                }
                aw(node.set_variables(v))
        for subgoal in path_goal:
            while not motion.check_robot_arrived(robot_pos,subgoal):
                captured, img_travel = cap.read()
                cropped, cropped_travel = vis.crop_map(img_travel)
                cv.imshow("cropped",cropped_travel)
                im4 = np.zeros(cropped_travel.shape)
                vis.annotate_robot(robot_pos,im4)
                vis.annotate_path(path_goal,im4)
                #print("robot angle", robot_pos[1])
                
                #read values from camera & sensors
                success_camera, robot_pos_camera = pos.get_robot_position(cropped_travel)
                if success_camera:
                    camera_input = [(robot_pos_camera[0][0],robot_pos_camera[0][1]), robot_pos_camera[1], success_camera]
                else:
                    camera_input = [(0,0),0,False]
                await node.wait_for_variables({"motor.left.speed"})
                await node.wait_for_variables({"motor.right.speed"})
                motorspeed = [node.v.motor.right.speed, node.v.motor.left.speed]        
                                
                #get position from kalman filter
                robot_pos_kalman, Sigma_kalman, history_kalman =\
                kalman.kalmanfilter(robot_pos_kalman,Sigma_kalman,motorspeed,history_kalman, camera_input, Ts, scale_factor, AXLE_LENGTH)
                robot_pos = [(int(robot_pos_kalman[0][0]), int(robot_pos_kalman[1][0])), robot_pos_kalman[2][0]]
                
                
                err = motion.get_error(robot_pos,subgoal)
                cv.putText(im4, str(err), (40, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv.imshow("mission status", im4)
                cv.waitKey(10)
                motor_left, motor_right = motion.speed_control(err)
                motor_left = round(motor_left)
                motor_right = round(motor_right)
                v = {
                    "motor.left.target": [motor_left],
                    "motor.right.target": [motor_right],
                }
                
                aw(node.set_variables(v))
        state = FINISH
    if state == FINISH:
        v = {
                    "motor.left.target": [0],
                    "motor.right.target": [0],
                }
        aw(node.set_variables(v))
            #print("motor_left",motor_left)
            #print("motor_right", motor_right)
            #motor_left_target = int(motor_left)
            #motor_right_target = int(motor_right)
            #print("motor_left_t",motor_left_target)
            #print("motor_right_t", motor_right_target)          
    cv.imshow("environment",im3)
    #cv.waitKey(0)
cap.release()
print("finished!")
# cv.destroyAllWindows()
    

"""
# robot run    
elif short.find_shortest_path() == True:
    state = 1

obstacle = check_obstacle(prox_horizontal)
    
        # global navigation
        if  obstacle == False:
            motion_control

        # local navigation
        elif obstacle == True:
"""
            
            
        
    

C:\Users\sebas\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:804: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, order=order, subok=subok, copy=True)


Start (506, 176)
Goal (178, 440)
Vertices [[(352, 354), (501, 313), (527, 448), (475, 452)], [(203, 290), (238, 337), (130, 474), (66, 487), (33, 445)], [(325, 147), (395, 266), (293, 328), (222, 206)], [(140, 96), (155, 231), (38, 245), (25, 113)], [(99, 19), (275, 48), (179, 135)]]
list neighbours [[(1, 235.3720459187964), (2, 137.09121051329294), (3, 272.8094573140748), (10, 183.30848316430968), (11, 142.90206436577463), (19, 264.09278672466615)], [(0, 235.3720459187964), (2, 154.53802121160993), (4, 157.26728839781018), (5, 162.16349774224778), (6, 115.26057435220423), (7, 252.3568901377571), (11, 97.94386147176351), (12, 64.47480127925948), (21, 194.09276132818556)], [(0, 137.09121051329294), (1, 154.53802121160993), (3, 137.48090776540573), (6, 264.09278672466615), (10, 241.93387526346947), (11, 115.95257651298655), (12, 208.54016399725018), (19, 348.28293096274473)], [(0, 272.8094573140748), (2, 137.48090776540573), (4, 52.15361924162119), (8, 462.6467334803089)], [(1, 157.26728

seen by camera [(504, 176), 0.35299038782691045, True]
robot_pos [(476, 176), 0.3529903878269103]
kalman[0][0] 476.2943577111706
kalman[1][0] 176.0
kalman[2][0] 0.3529903878269103
robot_pos [(476, 176), 0.3529903878269103]
kalman[0][0] 476.2943577111706
kalman[1][0] 176.0
kalman[2][0] 0.3529903878269103
seen by camera [(506, 174), 0.04995839572194276, True]
robot_pos [(505, 162), 0.05037460696607665]
kalman[0][0] 505.48825415296164
kalman[1][0] 162.64450826301191
kalman[2][0] 0.05037460696607665
seen by camera [(506, 174), -0.14888994760949725, True]
robot_pos [(505, 162), -0.27788855092865994]
kalman[0][0] 505.27188587150846
kalman[1][0] 162.58278495476458
kalman[2][0] -0.27788855092865994
robot_pos [(505, 162), -0.606151708823397]
kalman[0][0] 505.4568013349036
kalman[1][0] 162.71096947566062
kalman[2][0] -0.606151708823397
seen by camera [(508, 172), -0.5743048301747018, True]
robot_pos [(507, 166), -0.5748202801659499]
kalman[0][0] 507.87800458659865
kalman[1][0] 166.43319291712308

seen by camera [(442, 250), -2.5829933382462307, True]
robot_pos [(441, 252), -2.582764885247226]
kalman[0][0] 441.1893874607929
kalman[1][0] 252.8229442758435
kalman[2][0] -2.582764885247226
eureka en grand
seen by camera [(442, 252), -2.65711472455277, True]
robot_pos [(447, 251), -2.6570688209602285]
kalman[0][0] 447.66358174650554
kalman[1][0] 251.80575211249356
kalman[2][0] -2.6570688209602285
seen by camera [(438, 256), -2.677945044588987, True]
robot_pos [(439, 255), -2.67804125924654]
kalman[0][0] 439.62477145963805
kalman[1][0] 255.39968125921516
kalman[2][0] -2.67804125924654
robot_pos [(432, 259), -2.6581465224044347]
kalman[0][0] 432.45304516953126
kalman[1][0] 259.1648295988946
kalman[2][0] -2.6581465224044347
seen by camera [(426, 262), -2.654693421778524, True]
robot_pos [(425, 261), -2.6547661579830253]
kalman[0][0] 425.97771885958207
kalman[1][0] 261.6589314412274
kalman[2][0] -2.6547661579830253
seen by camera [(422, 264), -2.609868586330988, True]
robot_pos [(421, 26

eureka en grand
eureka en grand
